# Supervised sentiment: hand-built feature functions

In [ ]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2022"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Feature functions](#Feature-functions)
    1. [Unigrams](#Unigrams)
    1. [Bigrams](#Bigrams)
    1. [A note on DictVectorizer](#A-note-on-DictVectorizer)
1. [Building datasets for experiments](#Building-datasets-for-experiments)
1. [Basic optimization](#Basic-optimization)
    1. [Wrapper for SGDClassifier](#Wrapper-for-SGDClassifier)
    1. [Wrapper for LogisticRegression](#Wrapper-for-LogisticRegression)
    1. [Wrapper for TorchShallowNeuralClassifier](#Wrapper-for-TorchShallowNeuralClassifier)
    1. [A softmax classifier in PyTorch](#A-softmax-classifier-in-PyTorch)
    1. [Using sklearn Pipelines](#Using-sklearn-Pipelines)
1. [Hyperparameter search](#Hyperparameter-search)
    1. [utils.fit_classifier_with_hyperparameter_search](#utils.fit_classifier_with_hyperparameter_search)
    1. [Example using LogisticRegression](#Example-using-LogisticRegression)
1. [Reproducing  baselines from Socher et al. 2013](#Reproducing--baselines-from-Socher-et-al.-2013)
    1. [Reproducing the Unigram NaiveBayes results](#Reproducing-the-Unigram-NaiveBayes-results)
    1. [Reproducing the Bigrams NaiveBayes results](#Reproducing-the-Bigrams-NaiveBayes-results)
    1. [Reproducing the SVM results](#Reproducing-the-SVM-results)
1. [Statistical comparison of classifier models](#Statistical-comparison-of-classifier-models)
    1. [Comparison with the Wilcoxon signed-rank test](#Comparison-with-the-Wilcoxon-signed-rank-test)
    1. [Comparison with McNemar's test](#Comparison-with-McNemar's-test)

## Overview

* The focus of this notebook is __building feature representations__ for use with (mostly linear) classifiers (though you're encouraged to try out some non-linear ones as well!).

* The core characteristics of the feature functions we'll build here:
   * They represent examples in __very large, very sparse feature spaces__.
   * The individual feature functions can be __highly refined__, drawing on expert human knowledge of the domain. 
   * Taken together, these representations don't comprehensively represent the input examples. They just identify aspects of the inputs that the classifier model can make good use of (we hope).
   
* These classifiers tend to be __highly competitive__. We'll look at more powerful deep learning models in the next notebook, and it will immediately become apparent that it is very difficult to get them to measure up to well-built classifiers based in sparse feature representations. It can be done, but it tends to require a lot of attention to optimization details (and potentially a lot of compute resources).

* For this notebook, we look in detail at just two very general strategies for featurization: unigram-based and bigram-based. This gives us a chance to introduce core concepts in optimization. The [associated homework](hw_sst.ipynb) is oriented towards designing more specialized, linguistically intricate feature functions.

## Set-up

See [the previous notebook](sst_01_overview.ipynb#Set-up) for set-up instructions.

In [2]:
from collections import Counter
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import PredefinedSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import scipy.stats
import torch.nn as nn

from np_sgd_classifier import BasicSGDClassifier
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
import sst
import utils

In [3]:
utils.fix_random_seeds()

In [4]:
SST_HOME = os.path.join('data', 'sentiment')

## Feature functions

* Feature representation is arguably __the most important step in any machine learning task__. As you experiment with the SST, you'll come to appreciate this fact, since your choice of feature function will have a far greater impact on the effectiveness of your models than any other choice you make. This is especially true if you are careful to optimize the hyperparameters of your models.

* We will define our feature functions as `dict`s mapping feature names (which can be any object that can be a `dict` key) to their values (which must be `bool`, `int`, or `float`). 

* To prepare for optimization, we will use `sklearn`'s [DictVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) class to turn these into matrices of features. 

* The `dict`-based approach gives us a lot of flexibility and frees us from having to worry about the underlying feature matrix.

### Unigrams

A typical baseline or default feature representation in NLP or NLU is built from unigrams. Here, those are the leaf nodes of the tree:

In [5]:
def unigrams_phi(text):
    """
    The basis for a unigrams feature function. Downcases all tokens.

    Parameters
    ----------
    text : str
        The example to represent.

    Returns
    -------
    defaultdict
        A map from strings to their counts in `text`. (Counter maps a
        list to a dict of counts of the elements in that list.)

    """
    return Counter(text.lower().split())

In [6]:
example_text = "NLU is enlightening !"

In [7]:
unigrams_phi(example_text)

Counter({'nlu': 1, 'is': 1, 'enlightening': 1, '!': 1})

### Bigrams

In [8]:
def bigrams_phi(text):
    """
    The basis for a bigrams feature function. Downcases all tokens.

    Parameters
    ----------
    text : str
        The example to represent.

    Returns
    -------
    defaultdict
        A map from tuples to their counts in `text`.

    """
    toks = text.lower().split()
    left = [utils.START_SYMBOL] + toks
    right = toks + [utils.END_SYMBOL]
    grams = list(zip(left, right))
    return Counter(grams)

In [9]:
bigrams_phi(example_text)

Counter({('<s>', 'nlu'): 1,
         ('nlu', 'is'): 1,
         ('is', 'enlightening'): 1,
         ('enlightening', '!'): 1,
         ('!', '</s>'): 1})

It's generally good design to __write lots of atomic feature functions__ and then bring them together into a single function when running experiments. This will lead to reusable parts that you can assess independently and in sub-groups as part of development.

### A note on DictVectorizer

I've tried to be careful above to say that the above functions are just the __basis__ for feature representations. In truth, our models typically don't represent examples as dictionaries, but rather as vectors embedded in a matrix. In general, to manage the translation from dictionaries to vectors, we use  `sklearn.feature_extraction.DictVectorizer` instances. Here's a brief overview of how these work:

To start, suppose that we had just two examples to represent, and our feature function mapped them to the following list of dictionaries:

In [10]:
train_feats = [
    {'a': 1, 'b': 1},
    {'b': 1, 'c': 2}]

Now we create a `DictVectorizer`. So that we can more easily inspect the resulting matrix, I've set `sparse=False`, so that the return value is a dense matrix. For real problems, you'll probably want to use `sparse=True`, as it will be vastly more efficient for the very sparse feature matrices that you are likely to be creating.

In [11]:
vec = DictVectorizer(sparse=False)  # Use `sparse=True` for real problems!

The `fit_transform` method maps our list of dictionaries to a matrix:

In [12]:
X_train = vec.fit_transform(train_feats)

Here I'll create a `pd.Datafame` just to help us inspect `X_train`:

In [13]:
pd.DataFrame(X_train, columns=vec.get_feature_names())

,a,b,c
0,1.0,1.0,0.0
1,0.0,1.0,2.0


Now we can see that, intuitively, the feature called "a" is embedded in the first column, "b" in the second column, and "c" in the third.

Now suppose we have some new test examples:

In [14]:
test_feats = [
    {'a': 2, 'c': 1},
    {'a': 4, 'b': 2, 'd': 1}]

If we have trained a model on `X_train`, then it will not have any way to deal with this new feature "d". This shows that we need to embed `test_feats` in the same space as `X_train`. To do this, one just calls `transform` on the existing vectorizer:

In [15]:
X_test = vec.transform(test_feats)  # Not `fit_transform`!

In [16]:
pd.DataFrame(X_test, columns=vec.get_feature_names())

,a,b,c
0,2.0,0.0,1.0
1,4.0,2.0,0.0


The most common mistake with `DictVectorizer` is calling `fit_transform` on test examples. This will wipe out the existing representation scheme, replacing it with one that matches the test examples. That will happen silently, but then you'll find that the new representations are incompatible with the model you fit. This is likely to manifest itself as a `ValueError` relating to feature counts. Here's an example that might help you spot this if and when it arises in your own work: 

In [17]:
toy_mod = LogisticRegression()

vec = DictVectorizer(sparse=False)

X_train = vec.fit_transform(train_feats)

toy_mod.fit(X_train, [0, 1])

# Here's the error! Don't use `fit_transform` again! Use `transform`!
X_test = vec.fit_transform(test_feats)

try:
    toy_mod.predict(X_test)
except ValueError as err:
    print("ValueError: {}".format(err))

ValueError: X has 4 features per sample; expecting 3


This is actually the lucky case. If your train and test sets have the same number of features (columns), then no error will arise, and you might not even notice the misalignment between the train and test feature matrices.

In what follows, all these steps will be taken care of "under the hood", but it's good to be aware of what is happening. I think this also helps show the value of writing general experiment code, so that you don't have to check each experiment individually to make sure that you called the `DictVectorizer` methods (among other things!) correctly.

## Building datasets for experiments

The second major phase for our analysis is a kind of set-up phase. Ingredients:

* A dataset from a function like `sst.train_reader`
* A feature function like `unigrams_phi`

The convenience function `sst.build_dataset` uses these to build a dataset for training and assessing a model. See its documentation for details on how it works. Much of this is about taking advantage of `sklearn`'s many functions for model building.

In [18]:
train_dataset = sst.build_dataset(
    sst.train_reader(SST_HOME),
    phi=unigrams_phi,
    vectorizer=None)

In [19]:
print("Train dataset with unigram features has {:,} examples and "
      "{:,} features.".format(*train_dataset['X'].shape))

Train dataset with unigram features has 8,544 examples and 16,579 features.


Notice that `sst.build_dataset` has an optional argument `vectorizer`:

* If it is `None`, then a new vectorizer is used and returned as `dataset['vectorizer']`. This is the usual scenario when training. 

* For evaluation, one wants to represent examples exactly as they were represented during training. To ensure that this happens, pass the training `vectorizer` to this function, so that `transform` is used, [as discussed just above](#A-note-on-DictVectorizer).

In [20]:
dev_dataset = sst.build_dataset(
    sst.dev_reader(SST_HOME),
    phi=unigrams_phi,
    vectorizer=train_dataset['vectorizer'])

In [21]:
print("Dev dataset with unigram features has {:,} examples "
      "and {:,} features".format(*dev_dataset['X'].shape))

Dev dataset with unigram features has 1,101 examples and 16,579 features


## Basic optimization

We're now in a position to begin training supervised models!

For the most part, in this course, we will not study the theoretical aspects of machine learning optimization, concentrating instead on how to optimize systems effectively in practice. That is, this isn't a theory course, but rather an experimental, project-oriented one.

Nonetheless, we do want to avoid treating our optimizers as black boxes that work their magic and give us some assessment figures for whatever we feed into them. That seems irresponsible from a scientific and engineering perspective, and it also sends the false signal that the optimization process is inherently mysterious. So we do want to take a minute to demystify it with some simple code.

The module `np_sgd_classifier` contains a complete optimization framework, as `BasicSGDClassifier`. Well, it's complete in the sense that it achieves our full task of supervised learning. It's incomplete in the sense that it is very basic. You probably wouldn't want to use it in experiments. Rather, we're going to encourage you to rely on `sklearn` for your experiments (see below). Still, this is a good basic picture of what's happening under the hood.

So what is `BasicSGDClassifier` doing? The heart of it is the `fit` function (reflecting the usual `sklearn` naming system). This method implements a hinge-loss stochastic sub-gradient descent optimization. Intuitively, it works as follows:

1. Start by assuming that all the feature weights are `0`.
1. Move through the dataset instance-by-instance in random order.
1. For each instance, classify it using the current weights. 
1. If the classification is incorrect, move the weights in the direction of the correct classification

This process repeats for a user-specified number of iterations (default `10` below), and the weight movement is tempered by a learning-rate parameter `eta` (default `0.1`). The output is a set of weights that can be used to make predictions about new (properly featurized) examples.

In more technical terms, the objective function is 

$$
  \min_{\mathbf{w} \in \mathbb{R}^{d}}
  \sum_{(x,y)\in\mathcal{D}} 
  \max_{y'\in\mathbf{Y}}
  \left[\mathbf{Score}_{\textbf{w}, \phi}(x,y') + \mathbf{cost}(y,y')\right] - \mathbf{Score}_{\textbf{w}, \phi}(x,y)
$$

where $\mathbf{w}$ is the set of weights to be learned, $\mathcal{D}$ is the training set of example&ndash;label pairs, $\mathbf{Y}$ is the set of labels, $\mathbf{cost}(y,y') = 0$ if $y=y'$, else $1$, and $\mathbf{Score}_{\textbf{w}, \phi}(x,y')$ is the inner product of the weights 
$\mathbf{w}$ and the example as featurized according to $\phi$.

The `fit` method is then calculating the sub-gradient of this objective. In succinct pseudo-code:

* Initialize $\mathbf{w} = \mathbf{0}$
* Repeat $T$ times:
    * for each $(x,y) \in \mathcal{D}$ (in random order):
        * $\tilde{y} = \text{argmax}_{y'\in \mathcal{Y}} \mathbf{Score}_{\textbf{w}, \phi}(x,y') + \mathbf{cost}(y,y')$
        * $\mathbf{w} =  \mathbf{w} + \eta(\phi(x,y) - \phi(x,\tilde{y}))$
        
This is very intuitive – push the weights in the direction of the positive cases. It doesn't require any probability theory. And such loss functions have proven highly effective in many settings. For a more powerful version of this classifier, see [sklearn.linear_model.SGDClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier). With `loss='hinge'`, it should behave much like `BasicSGDClassifier` (but faster!).

For the most part, the classifiers that we use in this course have a softmax objective function. The module [np_shallow_neural_classifier.py](np_shallow_neural_classifier.py) is a straightforward example. The precise calculations are a bit less transparent than those for `BasicSGDClassifier`, but the general logic is the same for both.

### Wrapper for SGDClassifier

For the sake of our experimental framework, a simple wrapper for `SGDClassifier`:

In [22]:
def fit_basic_sgd_classifier(X, y):
    """
    Wrapper for `BasicSGDClassifier`.

    Parameters
    ----------
    X : np.array, shape `(n_examples, n_features)`
        The matrix of features, one example per row.

    y : list
        The list of labels for rows in `X`.

    Returns
    -------
    BasicSGDClassifier
        A trained `BasicSGDClassifier` instance.

    """
    mod = BasicSGDClassifier()
    mod.fit(X, y)
    return mod

This might look like a roundabout way of just calling `fit`. We'll see shortly that having a "wrapper" like this creates space for us to include a lot of other modeling steps.

We now have all the pieces needed to run experiments. And __we're going to want to run a lot of experiments__, trying out different feature functions, taking different perspectives on the data and labels, and using different models. 

To make that process efficient and regimented, `sst` contains a function `experiment`. All it does is pull together these pieces and use them for training and assessment. It's complicated, but the flexibility will turn out to be an asset. Here's an example with all of the default values spelled out:

In [23]:
_ = sst.experiment(
    sst.train_reader(SST_HOME),
    unigrams_phi,
    fit_basic_sgd_classifier,
    assess_dataframes=sst.dev_reader(SST_HOME),
    train_size=0.7,
    score_func=utils.safe_macro_f1,
    verbose=True)

              precision    recall  f1-score   support

    negative      0.660     0.526     0.585       428
     neutral      0.261     0.258     0.259       229
    positive      0.612     0.736     0.669       444

    accuracy                          0.555      1101
   macro avg      0.511     0.507     0.504      1101
weighted avg      0.558     0.555     0.551      1101



A few notes on this function call:
    
* Since `assess_dataframes=None`, the function reports performance on a random train–test split from `train_dataframes`, as given by the first argument. Give `sst.dev_reader(SST_HOME)` as the argument to assess against the `dev` set.

* `unigrams_phi` is the function we defined above. By changing/expanding this function, you can start to improve on the above baseline, perhaps periodically seeing how you do on the dev set.

* `fit_basic_sgd_classifier` is the wrapper we defined above. To assess new models, simply define more functions like this one. Such functions just need to consume an `(X, y)` pair constituting a dataset and return a model.

### Wrapper for LogisticRegression

As I said above, we likely don't want to rely on `BasicSGDClassifier` (though it does a good job with SST!). Instead, we want to rely on `sklearn` and our `torch_*` models. 

Here's a simple wrapper for [sklearn.linear.model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html):

In [24]:
def fit_softmax_classifier(X, y):
    """
    Wrapper for `sklearn.linear.model.LogisticRegression`. This is
    also called a Maximum Entropy (MaxEnt) Classifier, which is more
    fitting for the multiclass case.

    Parameters
    ----------
    X : np.array, shape `(n_examples, n_features)`
        The matrix of features, one example per row.

    y : list
        The list of labels for rows in `X`.

    Returns
    -------
    sklearn.linear.model.LogisticRegression
        A trained `LogisticRegression` instance.

    """
    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='auto')
    mod.fit(X, y)
    return mod

And an experiment using `fit_softmax_classifier` and `unigrams_phi`:

In [25]:
_ = sst.experiment(
    sst.train_reader(SST_HOME),
    unigrams_phi,
    fit_softmax_classifier)

              precision    recall  f1-score   support

    negative      0.628     0.689     0.657       996
     neutral      0.336     0.157     0.214       479
    positive      0.671     0.770     0.717      1089

    accuracy                          0.624      2564
   macro avg      0.545     0.538     0.529      2564
weighted avg      0.592     0.624     0.600      2564



### Wrapper for TorchShallowNeuralClassifier

While we're at it, we might as well start to get a sense for whether adding a hidden layer to our softmax classifier yields any benefits. Whereas `LogisticRegression` is, at its core, computing

$$\begin{align*}
y &= \textbf{softmax}(xW_{xy} + b_{y})
\end{align*}$$

the shallow neural network inserts a hidden layer with a non-linear activation applied to it:

$$\begin{align*}
h &= \tanh(xW_{xh} + b_{h}) \\
y &= \textbf{softmax}(hW_{hy} + b_{y})
\end{align*}$$

Here's an illustrative example using `TorchShallowNeuralClassifier`, which is in the course repo's `torch_shallow_neural_classifier.py`:

In [26]:
def fit_nn_classifier(X, y):
    mod = TorchShallowNeuralClassifier(
        hidden_dim=100,
        early_stopping=True,      # A basic early stopping set-up.
        validation_fraction=0.1,  # If no improvement on the
        tol=1e-5,                 # validation set is seen within
        n_iter_no_change=10)      # `n_iter_no_change`, we stop.
    mod.fit(X, y)
    return mod

A noteworthy feature of this `fit_nn_classifier` is that it sets `early_stopping=True`. This instructs the optimizer to hold out a small fraction (see `validation_fraction`) of the training data to use as a dev set at the end of each epoch. Optimization will stop if improvements of at least `tol` on this dev set aren't seen within `n_iter_no_change` epochs. If that condition is triggered, the parameters from the top-scoring model are used for the final model. (For additional discussion, see [the section on model convergence in the evaluation methods notebook](#Assessing-models-without-convergence).)

Another quick experiment:

In [27]:
_ = sst.experiment(
    sst.train_reader(SST_HOME),
    unigrams_phi,
    fit_nn_classifier)

Stopping after epoch 23. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 0.7719634175300598

              precision    recall  f1-score   support

    negative      0.627     0.714     0.668       977
     neutral      0.321     0.105     0.158       497
    positive      0.659     0.779     0.714      1090

    accuracy                          0.624      2564
   macro avg      0.536     0.533     0.513      2564
weighted avg      0.581     0.624     0.589      2564



### A softmax classifier in PyTorch

Our PyTorch modules should support easy modification as discussed in [tutorial_pytorch_models.ipynb](tutorial_pytorch_models.ipynb). Perhaps the simplest modification from that notebook uses `TorchShallowNeuralClassifier` to define a `TorchSoftmaxClassifier`. All you need to do for this is write a new `build_graph` method:

In [28]:
class TorchSoftmaxClassifier(TorchShallowNeuralClassifier):

    def build_graph(self):
        return nn.Linear(self.input_dim, self.n_classes_)

For this function call, I added an L2 regularization term to help prevent overfitting:

In [29]:
def fit_torch_softmax(X, y):
    mod = TorchSoftmaxClassifier(l2_strength=0.0001)
    mod.fit(X, y)
    return mod

In [30]:
_ = sst.experiment(
    sst.train_reader(SST_HOME),
    unigrams_phi,
    fit_torch_softmax)

Stopping after epoch 609. Training loss did not improve more than tol=1e-05. Final error is 1.1642730385065079.

              precision    recall  f1-score   support

    negative      0.633     0.691     0.661       969
     neutral      0.317     0.174     0.225       528
    positive      0.661     0.753     0.704      1067

    accuracy                          0.610      2564
   macro avg      0.537     0.539     0.530      2564
weighted avg      0.579     0.610     0.589      2564



### Using sklearn Pipelines

The `sklearn.pipeline` module defines `Pipeline` objects, which let you chain together different transformations and estimators. `Pipeline` objects are fully compatible with `sst.experiment`. Here's a basic example using `TfidfTransformer` followed by `LogisticRegression`:

In [31]:
def fit_pipeline_softmax(X, y):
    rescaler = TfidfTransformer()
    mod = LogisticRegression(max_iter=2000)
    pipeline = Pipeline([
        ('scaler', rescaler),
        ('model', mod)])
    pipeline.fit(X, y)
    return pipeline

In [32]:
_ = sst.experiment(
    sst.train_reader(SST_HOME),
    unigrams_phi,
    fit_pipeline_softmax)

              precision    recall  f1-score   support

    negative      0.611     0.726     0.664       965
     neutral      0.361     0.051     0.089       511
    positive      0.646     0.798     0.714      1088

    accuracy                          0.622      2564
   macro avg      0.539     0.525     0.489      2564
weighted avg      0.576     0.622     0.570      2564



Pipelines can also include the models from the course repo. The one gotcha here is that some `sklearn` transformers return sparse matrices, which are likely to clash with the requirements of these other models. To get around this, just add `utils.DenseTransformer()` where you need to transition from a sparse matrix to a dense one. Here's an example using `TorchShallowNeuralClassifier` with early stopping:

In [33]:
def fit_pipeline_classifier(X, y):
    rescaler = TfidfTransformer()
    mod = TorchShallowNeuralClassifier(early_stopping=True)
    pipeline = Pipeline([
        ('scaler', rescaler),
        # We need this little bridge to go from
        # sparse matrices to dense ones:
        ('densify', utils.DenseTransformer()),
        ('model', mod)])
    pipeline.fit(X, y)
    return pipeline

In [34]:
_ = sst.experiment(
    sst.train_reader(SST_HOME),
    unigrams_phi,
    fit_pipeline_classifier)

Stopping after epoch 13. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 4.746284365653992

              precision    recall  f1-score   support

    negative      0.588     0.700     0.639       997
     neutral      0.239     0.274     0.256       456
    positive      0.740     0.569     0.643      1111

    accuracy                          0.567      2564
   macro avg      0.522     0.514     0.513      2564
weighted avg      0.592     0.567     0.573      2564



## Hyperparameter search

The training process learns __parameters__ &mdash; the weights. There are typically lots of other parameters that need to be set. For instance, our `BasicSGDClassifier` has a learning rate parameter and a training iteration parameter. These are called __hyperparameters__. The more powerful `sklearn` classifiers and our `torch_*` models have many more such hyperparameters. These are outside of the explicitly stated objective, hence the "hyper" part. 

So far, we have just set the hyperparameters by hand. However, their optimal values can vary widely between datasets, and choices here can dramatically impact performance, so we would like to set them as part of the overall experimental framework.

### utils.fit_classifier_with_hyperparameter_search

Luckily, `sklearn` provides a lot of functionality for setting hyperparameters via cross-validation. The function `utils.fit_classifier_with_hyperparameter_search` implements a basic framework for taking advantage of these options. It's really just a lightweight wrapper around `slearn.model_selection.GridSearchCV`.

This corresponding model wrappers have the same basic shape as `fit_softmax_classifier` above: they take a dataset as input and return a trained model. However, to find the best model, they explore a space of hyperparameters supplied by the user, seeking the optimal combination of settings. 

Only the training data is used to perform this search; that data is split into multiple train–test splits, and the best hyperparameter settings are the one that do the best on average across these splits. Once those settings are found, a model is trained with those settings on all the available data and finally evaluated on the assessment data.

### Example using LogisticRegression

Here's a fairly full-featured use of the above for the `LogisticRegression` model family:

In [35]:
def fit_softmax_with_hyperparameter_search(X, y):
    """
    A MaxEnt model of dataset with hyperparameter cross-validation.

    Some notes:

    * 'fit_intercept': whether to include the class bias feature.
    * 'C': weight for the regularization term (smaller is more regularized).
    * 'penalty': type of regularization -- roughly, 'l1' ecourages small
      sparse models, and 'l2' encourages the weights to conform to a
      gaussian prior distribution.
    * 'class_weight': 'balanced' adjusts the weights to simulate a
      balanced class distribution, whereas None makes no adjustment.

    Other arguments can be cross-validated; see
    http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

    Parameters
    ----------
    X : 2d np.array
        The matrix of features, one example per row.

    y : list
        The list of labels for rows in `X`.

    Returns
    -------
    sklearn.linear_model.LogisticRegression
        A trained model instance, the best model found.

    """
    basemod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='auto')
    cv = 5
    param_grid = {
        'C': [0.6, 0.8, 1.0, 2.0],
        'penalty': ['l1', 'l2'],
        'class_weight': ['balanced', None]}
    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, basemod, cv, param_grid)
    return bestmod

In [36]:
softmax_experiment = sst.experiment(
    sst.train_reader(SST_HOME),
    unigrams_phi,
    fit_softmax_with_hyperparameter_search,
    assess_dataframes=sst.dev_reader(SST_HOME))

Best params: {'C': 1.0, 'class_weight': 'balanced', 'penalty': 'l2'}
Best score: 0.541
              precision    recall  f1-score   support

    negative      0.635     0.659     0.647       428
     neutral      0.313     0.223     0.260       229
    positive      0.652     0.725     0.687       444

    accuracy                          0.595      1101
   macro avg      0.533     0.536     0.531      1101
weighted avg      0.575     0.595     0.582      1101



Recall that the "Best params" are found via evaluations only on the training data. The `assess_reader` is held out from that process, so it's giving us an estimate of how we will do on a final test set.

## Reproducing  baselines from Socher et al. 2013

The goal of this section is to bring together ideas from the above to reproduce some of the the non-neural baselines from [Socher et al., Table 1](http://www.aclweb.org/anthology/D/D13/D13-1170.pdf). More specifically, we'll shoot for the root-level binary numbers:

| Model              | Accuracy  | 
|--------------------|-----------|
| Unigram NaiveBayes | 81.8      |
| Bigram NaiveBayes  | 83.1      |
| SVM                | 79.4      |

The following reduces the dataset to the binary task:

In [37]:
train_df = sst.train_reader(SST_HOME)

train_bin_df = train_df[train_df.label != 'neutral']

In [38]:
dev_df = sst.dev_reader(SST_HOME)

dev_bin_df = dev_df[dev_df.label != 'neutral']

In [39]:
test_df = sst.sentiment_reader(os.path.join(SST_HOME, "sst3-test-labeled.csv"))

test_bin_df = test_df[test_df.label != 'neutral']

Note: we will continue to train on just the full examples, so that the experiments do not require a lot of time and computational resources. However, there are probably gains to be had from training on the subtrees as well. In that case, one needs to be careful in cross-validation: the test set needs to be the root-only dev set rather than slices of the train set. To achieve this, one can use a `PredefinedSplit`:

In [40]:
full_train_df = sst.train_reader(SST_HOME, include_subtrees=True)
full_train_bin_df = full_train_df[full_train_df.label != 'neutral']

split_indices = [0] * full_train_bin_df.shape[0]
split_indices += [-1] * dev_bin_df.shape[0]
sst_train_dev_splitter = PredefinedSplit(split_indices)

This would be used in place of `cv=5` in the model wrappers below.

### Reproducing the Unigram NaiveBayes results

To start, we might just use `MultinomialNB` with default parameters:

In [41]:
def fit_unigram_nb_classifier(X, y):
    mod = MultinomialNB()
    mod.fit(X, y)
    return mod

In [42]:
_ = sst.experiment(
    train_bin_df,
    unigrams_phi,
    fit_unigram_nb_classifier,
    assess_dataframes=dev_bin_df)

              precision    recall  f1-score   support

    negative      0.802     0.778     0.790       428
    positive      0.792     0.815     0.804       444

    accuracy                          0.797       872
   macro avg      0.797     0.797     0.797       872
weighted avg      0.797     0.797     0.797       872



This falls slightly short of our goal, which is not encouraging about how we would do on the test set. However, `MultinomialNB` has a regularization term `alpha` that might have a significant impact given the very large, sparse feature matrices we are creating with `unigrams_phi`. In addition, it might help to transform the raw feature counts, for the same reason that reweighting was so powerful in our VSM module. The best way to try out all these ideas is to do a wide hyperparameter search. The following model wrapper function implements these steps: 

In [43]:
def fit_nb_classifier_with_hyperparameter_search(X, y):
    rescaler = TfidfTransformer()
    mod = MultinomialNB()

    pipeline = Pipeline([('scaler', rescaler), ('model', mod)])

    # Access the alpha and fit_prior parameters of `mod` with
    # `model__alpha` and `model__fit_prior`, where "model" is the
    # name from the Pipeline. Use 'passthrough' to optionally
    # skip TF-IDF.
    param_grid = {
        'model__fit_prior': [True, False],
        'scaler': ['passthrough', rescaler],
        'model__alpha': [0.1, 0.2, 0.4, 0.8, 1.0, 1.2]}

    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, pipeline,
        param_grid=param_grid,
        cv=5)
    return bestmod

Then we run the experiment:

In [44]:
unigram_nb_experiment_xval = sst.experiment(
    [train_bin_df, dev_bin_df],
    unigrams_phi,
    fit_nb_classifier_with_hyperparameter_search,
    assess_dataframes=test_bin_df)

Best params: {'model__alpha': 1.2, 'model__fit_prior': False, 'scaler': TfidfTransformer()}
Best score: 0.798
              precision    recall  f1-score   support

    negative      0.852     0.798     0.824       912
    positive      0.810     0.861     0.835       909

    accuracy                          0.830      1821
   macro avg      0.831     0.830     0.830      1821
weighted avg      0.831     0.830     0.830      1821



We're above the target of 81.8, so we can say that we reproduced the paper's result.

### Reproducing the Bigrams NaiveBayes results

For the bigram NaiveBayes mode, we can continue to use `fit_nb_classifier_with_hyperparameter_search`, but now the experiment is done with `bigrams_phi`:

In [45]:
bigram_nb_experiment_xval = sst.experiment(
    [train_bin_df, dev_bin_df],
    bigrams_phi,
    fit_nb_classifier_with_hyperparameter_search,
    assess_dataframes=test_bin_df)

Best params: {'model__alpha': 0.2, 'model__fit_prior': False, 'scaler': TfidfTransformer()}
Best score: 0.764
              precision    recall  f1-score   support

    negative      0.796     0.754     0.775       912
    positive      0.766     0.806     0.786       909

    accuracy                          0.780      1821
   macro avg      0.781     0.780     0.780      1821
weighted avg      0.781     0.780     0.780      1821



This is below the target of 83.1, so we've failed to reproduce the paper's result. I am not sure where the implementation difference between our model and that of Socher et al. lies!

### Reproducing the SVM results

In [46]:
def fit_svm_classifier_with_hyperparameter_search(X, y):
    rescaler = TfidfTransformer()
    mod = LinearSVC(loss='squared_hinge', penalty='l2')

    pipeline = Pipeline([('scaler', rescaler), ('model', mod)])

    # Access the alpha parameter of `mod` with `mod__alpha`,
    # where "model" is the name from the Pipeline. Use
    # 'passthrough' to optionally skip TF-IDF.
    param_grid = {
        'scaler': ['passthrough', rescaler],
        'model__C': [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]}

    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, pipeline,
        param_grid=param_grid,
        cv=5)
    return bestmod

In [47]:
svm_experiment_xval = sst.experiment(
    [train_bin_df, dev_bin_df],
    unigrams_phi,
    fit_svm_classifier_with_hyperparameter_search,
    assess_dataframes=test_bin_df)

Best params: {'model__C': 0.4, 'scaler': TfidfTransformer()}
Best score: 0.797
              precision    recall  f1-score   support

    negative      0.844     0.795     0.819       912
    positive      0.806     0.853     0.828       909

    accuracy                          0.824      1821
   macro avg      0.825     0.824     0.824      1821
weighted avg      0.825     0.824     0.824      1821



Right on! This is quite a ways above the target of 79.4, so we can say that we successfully reproduced this result.

## Statistical comparison of classifier models

Suppose two classifiers differ according to an effectiveness measure like F1 or accuracy. Are they meaningfully different?

* For very large datasets, the answer might be clear: if performance is very stable across different train/assess splits and the difference in terms of correct predictions has practical importance, then you can clearly say yes. 

* With smaller datasets, or models whose performance is closer together, it can be harder to determine whether the two models are different. We can address this question in a basic way with repeated runs and basic null-hypothesis testing on the resulting score vectors.

In general, one wants to compare __two feature functions against the same model__, or one wants to compare __two models with the same feature function used for both__. If both are changed at the same time, then it will be hard to figure out what is causing any differences you see.

### Comparison with the Wilcoxon signed-rank test

The function `sst.compare_models` is designed for such testing. The default set-up uses the non-parametric [Wilcoxon signed-rank test](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) to make the comparisons, which is relatively conservative and recommended by [Demšar 2006](http://www.jmlr.org/papers/v7/demsar06a.html) for cases where one can afford to do multiple assessments. For discussion, see [the evaluation methods notebook](evaluation_methods.ipynb#Wilcoxon-signed-rank-test).

Here's an example showing the default parameters values and comparing `LogisticRegression` and `BasicSGDClassifier`:

In [48]:
_ = sst.compare_models(
    sst.train_reader(SST_HOME),
    unigrams_phi,
    fit_softmax_classifier,
    stats_test=scipy.stats.wilcoxon,
    trials=10,
    phi2=None,  # Defaults to same as first argument.
    train_func2=fit_basic_sgd_classifier, # Defaults to same as second argument.
    train_size=0.7,
    score_func=utils.safe_macro_f1)

Model 1 mean: 0.522
Model 2 mean: 0.509
p = 0.014


### Comparison with McNemar's test

[McNemar's test](https://en.wikipedia.org/wiki/McNemar%27s_test) operates directly on the vectors of predictions for the two models being compared. As such, it doesn't require repeated runs, which is good where optimization is expensive. For discussion, see [the evaluation methods notebook](evaluation_methods.ipynb#McNemar's-test).

In [50]:
m = utils.mcnemar(
    unigram_nb_experiment_xval['assess_datasets'][0]['y'],
    unigram_nb_experiment_xval['predictions'][0],
    bigram_nb_experiment_xval['predictions'][0])

In [51]:
p = "p < 0.0001" if m[1] < 0.0001 else m[1]

print("McNemar's test: {0:0.02f} ({1:})".format(m[0], p))

McNemar's test: 22.38 (p < 0.0001)
